In [1]:
import pandas as pd
import csv
import numpy as np
from umap.umap_ import UMAP
import sklearn
from sklearn.manifold import Isomap
import kmapper as km
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram

from scipy.stats import zscore
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter

import sys
from pathlib import Path

In [2]:
# Add the project directory to sys.path

project_root = Path.cwd().parent
project_dir = str(project_root)
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [3]:
from tda_genes_alzheimers.my_support_functions import *
from tda_genes_alzheimers.config import gene_expression_data_path, gene_meta_data_path, intermediate_data_dir

In [4]:
# gene metadata
F_gene_meta_df = pd.read_csv(gene_meta_data_path, sep='\t', comment='^', skiprows=27, low_memory=False) 
F_gene_meta_df = F_gene_meta_df.drop(F_gene_meta_df.index[-1])      # delete end of table row

# gene expression data
with open(gene_expression_data_path, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    GSE44772_list_data = list(reader)                                               # list to export meta_data
GSE44772_df = pd.read_csv(gene_expression_data_path, comment='!', sep ='\t')        # data frame for analysis
# subject meta
metadata_df = extract_metadata_GSE44772(GSE44772_list_data)
metadata_df = metadata_df[:-1]
num_metadata_df, category_mapping = convert_meta_to_numeric(metadata_df)
metadata_sorted_df = metadata_df.sort_values(by=['Disease'])    
num_metadata_sorted_df = num_metadata_df.sort_values(by=['Disease'])   

In [5]:
#load
path_to_load = intermediate_data_dir / 'preprocessed'
preprocessed_df = pd.read_csv(path_to_load/'pp_GSE44772.csv', sep='\t')
preprocessed_gene_meta_df = pd.read_csv(path_to_load/'pp_GSE44772_genes.csv', sep='\t') 

In [6]:
# sort to make things easier in the future
preprocessed_df.sort_values(by=['ID_REF'], inplace=True)
preprocessed_gene_meta_df.sort_values(by=['ID_REF'], inplace=True)
preprocessed_gene_meta_df.reset_index(drop=True, inplace=True)

In [7]:
# DLPFC
corr_H_DLPFC = get_correlation_for_group_BR(metadata_sorted_df, preprocessed_df, 'N', 'DLPFC')
corr_A_DLPFC = get_correlation_for_group_BR(metadata_sorted_df, preprocessed_df, 'A', 'DLPFC')



In [8]:
path_to_saved_maps = intermediate_data_dir / 'maps' / 'genes' /'BR_GO'      # Brain Region GO folder

# initialize mapper
mapper = km.KeplerMapper(verbose=1)
# mapper parameters 
BR_projection = [Isomap(n_components=100, n_jobs=-1), UMAP(n_components=2, random_state=1)]

# get gene symbols do display on mapper visualization (instead of gene IDs)
filtered_symbols_arr = get_gene_symbols_for_viz(F_gene_meta_df, preprocessed_gene_meta_df)

# intialize Vietoris Rips for PD
VR = VietorisRipsPersistence(homology_dimensions=[0, 1])

KeplerMapper(verbose=1)


In [19]:
%%capture

# H
projected_H_DLPFC =  mapper.fit_transform(corr_H_DLPFC, projection=BR_projection)
Graph_H_DLPFC = mapper.map(lens=projected_H_DLPFC, X=corr_H_DLPFC,
                    clusterer=sklearn.cluster.DBSCAN(metric="cosine"),
                    cover = km.Cover(n_cubes=80, perc_overlap=0.65))

# # A
projected_A_DLPFC =  mapper.fit_transform(corr_A_DLPFC, projection=BR_projection)
Graph_A_DLPFC = mapper.map(lens=projected_A_DLPFC, X=corr_A_DLPFC,
                    clusterer=sklearn.cluster.DBSCAN(metric="cosine"),
                    cover = km.Cover(n_cubes=80, perc_overlap=0.6))

# extracts DBSCAN colors (only for Healthy)
labels_DLPFC, colorscale_DLPFC = get_cluster_labels_and_colorscale_GSE33000(Graph_H_DLPFC, corr_H_DLPFC)


In [10]:
print(Graph_A_DLPFC.keys())

dict_keys(['nodes', 'links', 'simplices', 'meta_data', 'meta_nodes'])


In [25]:
Graph_A_DLPFC['nodes']['cube575_cluster0']

[567, 933, 1422, 1660, 1813, 2415, 5003, 5010]

In [11]:
for key, value in Graph_A_DLPFC.items():
    print(f"Key: {key} -> Value: {value}")


Key: nodes -> Value: defaultdict(<class 'list'>, {'cube1_cluster0': [1407, 1762, 2887, 3024, 3125, 3567, 4022], 'cube2_cluster0': [1407, 1762, 1792, 3024, 3125, 3567, 4022], 'cube3_cluster0': [542, 1370, 1792, 1926, 2597, 2634, 2671, 3291, 3740, 4216, 5304, 5558, 5620], 'cube4_cluster0': [542, 1370, 1792, 1926, 2263, 2597, 2634, 2671, 3291, 3740, 4216, 5304, 5558, 5620], 'cube5_cluster0': [1926, 2263, 2634, 2671, 3291, 5620], 'cube8_cluster0': [87, 863, 1688, 2297, 2377, 3199, 3411, 3716, 3801, 3982, 4066, 4642, 4865, 4903, 5137, 5260, 5474, 5562, 6356], 'cube9_cluster0': [87, 863, 1688, 2297, 2377, 3199, 3411, 3716, 3801, 3982, 4066, 4642, 4865, 4903, 5137, 5260, 5474, 5562, 6356], 'cube12_cluster0': [901, 1501, 2749, 3701, 4414, 5542, 5657], 'cube13_cluster0': [901, 1407, 1501, 2237, 2749, 2887, 3125, 3567, 3701, 4414, 5657], 'cube14_cluster0': [358, 1133, 1407, 1579, 1762, 2237, 2775, 2887, 3024, 3125, 3567, 3823, 3879, 4022, 4703, 5657], 'cube15_cluster0': [358, 749, 904, 1133, 140

In [23]:
for node_id, data_indices in Graph_A_DLPFC['nodes'].items():
    print(node_id, data_indices)

cube1_cluster0 [1407, 1762, 2887, 3024, 3125, 3567, 4022]
cube2_cluster0 [1407, 1762, 1792, 3024, 3125, 3567, 4022]
cube3_cluster0 [542, 1370, 1792, 1926, 2597, 2634, 2671, 3291, 3740, 4216, 5304, 5558, 5620]
cube4_cluster0 [542, 1370, 1792, 1926, 2263, 2597, 2634, 2671, 3291, 3740, 4216, 5304, 5558, 5620]
cube5_cluster0 [1926, 2263, 2634, 2671, 3291, 5620]
cube8_cluster0 [87, 863, 1688, 2297, 2377, 3199, 3411, 3716, 3801, 3982, 4066, 4642, 4865, 4903, 5137, 5260, 5474, 5562, 6356]
cube9_cluster0 [87, 863, 1688, 2297, 2377, 3199, 3411, 3716, 3801, 3982, 4066, 4642, 4865, 4903, 5137, 5260, 5474, 5562, 6356]
cube12_cluster0 [901, 1501, 2749, 3701, 4414, 5542, 5657]
cube13_cluster0 [901, 1407, 1501, 2237, 2749, 2887, 3125, 3567, 3701, 4414, 5657]
cube14_cluster0 [358, 1133, 1407, 1579, 1762, 2237, 2775, 2887, 3024, 3125, 3567, 3823, 3879, 4022, 4703, 5657]
cube15_cluster0 [358, 749, 904, 1133, 1407, 1579, 1762, 1792, 2237, 2775, 3024, 3125, 3567, 3730, 3823, 3879, 4022, 4172, 4703, 5345]


In [24]:
aggregation_functions = {
    "mean": np.mean,
    "max": np.max,
    "std": np.std,
    "median": np.median
}

# Prepare a DataFrame to store the node color results
node_color_data = []

# Iterate through each node in the graph
for node_id, data_indices in Graph_A_DLPFC['nodes'].items():
    # Extract the 'DBSCAN' values for the data points in this node
    node_values = labels_DLPFC[data_indices]


    # Calculate the node colors based on aggregation functions
    node_colors = {func_name: func(node_values) for func_name, func in aggregation_functions.items()}
    
    # Add node ID and calculated colors to the result
    node_colors['Node ID'] = node_id
    node_colors['Gene ID'] = data_indices
    node_color_data.append(node_colors)

# Create a DataFrame with the node color results
node_color_df = pd.DataFrame(node_color_data)

# Convert the index into a column
node_color_df.reset_index(inplace=True)

# Rename the new column from 'index' to 'Label'
node_color_df.rename(columns={'index': 'Label'}, inplace=True)


In [26]:
# Calculate the threshold for the top 20% of 'std' values
threshold = node_color_df['std'].quantile(0.80)

# Filter the DataFrame to get only the rows with 'std' values in the top 20%
top_20_percent_std_df = node_color_df[node_color_df['std'] >= threshold]


In [31]:
# Flatten the list of lists and count unique genes
unique_genes = set(gene for sublist in top_20_percent_std_df['Gene ID'] for gene in sublist)
unique_genes_count = len(unique_genes)

print(f"Number of unique genes: {unique_genes_count}")

Number of unique genes: 2112


In [28]:
# extract go: biological process, molecular function, cell component
go_bp, go_mf, go_cc = extract_go_terms(gene_meta_data_path)

In [30]:
pp_bp_df = get_go_for_existing_genes(preprocessed_gene_meta_df, 'GO:Process', F_gene_meta_df)

In [32]:
unique_genes_std_bp = pd.DataFrame(unique_genes)